In [1]:
import pandas as pd
import os

In [2]:
def load_google_map_dfs():
    base_path = os.getcwd()
    dfs = []
    column_names = ['latitude', 'longitude']
    
    file_names = ['North-Train-To-West-Left-Google-Maps-Route.csv',
                  'North-Train-To-West-Middle-Google-Maps-Route.csv',
                  'West-Train-To-North-Bottom-Google-Maps-Route.csv',
                  'West-Train-To-North-Middle-Google-Maps-Route.csv',
                  'West-Train-To-North-Top-Google-Maps-Route.csv'
                  ]

    for file_name in file_names:
        df = pd.read_csv(base_path + '/' + file_name)
        df.columns = column_names
        dfs.append(df)

    return dfs

In [3]:
google_maps_routes = load_google_map_dfs()

In [5]:
max_lat = 23.0
min_lat = 22.0

min_long = 113
max_long = 115

diff_in_latitude = max_lat - min_lat
diff_in_longitude = max_long - min_long


cell_size = diff_in_latitude / 20.0

In [6]:
def concat(row, col):
    return str(row) + '-' + str(col)


def map_gps_to_box(latitude, longitude):
    row_number = int((latitude - min_lat) // cell_size)
    col_number = int((longitude - min_long) // cell_size)

    if col_number < 0 or row_number < 0:
        return -1, -1, -1

    cell_number_str = str(row_number) + str(col_number)

    return concat(row_number, col_number), row_number, col_number

In [7]:
def map_gps_to_cell(df):
    cells = []
    rows = []
    cols = []

    for index, row in df.iterrows():
        lat = row['latitude']
        long = row['longitude']

        cell_number, cell_row, cell_col = map_gps_to_box(lat, long)

        cells.append(cell_number)
        cols.append(cell_col)
        rows.append(cell_row)

    df['cell'] = cells
    df['row'] = rows
    df['column'] = cols

    return df

In [8]:
def map_google_maps_routes_to_cells(df_list):
    with_cells = []
    for df in df_list:
        df = map_gps_to_cell(df)
        with_cells.append(df)
    return with_cells

In [9]:
google_maps_with_cells = map_google_maps_routes_to_cells(google_maps_routes)

In [12]:
def save_google_maps_routes_with_cells(route_df_list):
    file_names = ['North-Train-To-West-Left-Google-Maps-Route-Cells.csv',
                  'North-Train-To-West-Middle-Google-Maps-Route-Cells.csv',
                  'West-Train-To-North-Bottom-Google-Maps-Route-Cells.csv',
                  'West-Train-To-North-Middle-Google-Maps-Route-Cells.csv',
                  'West-Train-To-North-Top-Google-Maps-Route-Cells.csv'
                  ]

    for index, df in enumerate(route_df_list):
        df.to_csv(file_names[index], encoding='utf-8', index=False)
    
    return

In [13]:
save_google_maps_routes_with_cells(google_maps_with_cells)